Here's an extensive summary in markdown format, drawing on the provided sources and our conversation history, with some bolding to highlight key concepts:

**Debugging LLM Applications in LangChain**

*   **The Need for Debugging**: When building applications with Large Language Models (LLMs), debugging is essential. Issues can arise from various points in the application including failed model calls, misformatted outputs, and errors in nested model calls.
*   **Debugging Methods**: LangChain provides three primary methods for debugging:
    *   **Verbose Mode**: Adds print statements for key events in a chain, offering a basic level of visibility.
    *   **Debug Mode**: Adds logging statements for *all* events within a chain, providing comprehensive detail.
    *  **LangSmith Tracing**: Logs events to LangSmith, allowing for visualisation and a deeper inspection of the application's inner workings.

*   **Comparison of Debugging Methods**:

| Feature          | Verbose Mode | Debug Mode | LangSmith Tracing |
| ---------------- | ------------ | ---------- | ----------------- |
| Free             | ✅          | ✅        | ✅                |
| UI               | ❌           | ❌        | ✅                |
| Persisted        | ❌           | ❌        | ✅                |
| See all events   | ❌           | ✅        | ✅                |
| See "important" events  | ✅        | ❌      | ✅                |
| Runs Locally     | ✅           | ✅        | ❌                |

*   **LangSmith Tracing**:
    *   As applications become more complex with multiple steps and LLM calls, tracing becomes crucial to understand the behaviour of chains or agents.
    *   LangSmith is the recommended tool for tracing.
    *   To start logging traces, you need to sign up for LangSmith and set environment variables:
        ```bash
        export LANGCHAIN_TRACING_V2="true"
        export LANGCHAIN_API_KEY="..."
        ```
    *   In a notebook, environment variables can be set like this:
        ```python
        import getpass
        import os
        os.environ["LANGCHAIN_TRACING_V2"] = "true"
        os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
        ```

*   **Example with an Agent**:
    *   The sources demonstrates using an agent to answer a question about a movie director and their age.
    *   Without debugging, the output is limited.
    *   With LangSmith enabled, detailed information about the agent's actions can be seen via a web browser.

*  **Verbose Mode (`set_verbose(True)`)**:
    *   For prototyping in Jupyter Notebooks or Python scripts, verbose mode is a good option for printing intermediate steps.
    *   Setting the verbose flag will print inputs and outputs in a readable format, while skipping raw outputs like token usage statistics.
    *   It still works with LangSmith, so both can be enabled simultaneously.
    *   Verbose mode will print the calls to tools, including the input and output.

*   **Debug Mode (`set_debug(True)`)**:
    *   Setting the debug flag will print inputs and outputs for all LangChain components (chains, models, agents, tools, retrievers).
    *   This mode is very verbose and logs all raw inputs and outputs.
    *   Debug mode prints all steps of the chain, from the initial input to the final output, as well as all intermediate steps.
        * This includes the start and end of chain runs, prompt runs, LLM runs, tool runs, and parser runs.
    *   It also shows the inputs and outputs of each component, including the prompts sent to the LLM and the results of tool calls.

*   **Combining Debugging Methods**: It is possible to use LangSmith tracing alongside the local verbose or debug modes.

*   **Key Takeaways**
    *   Debugging is essential when building LLM applications due to potential issues like failed model calls or misformatted outputs.
    *  LangChain offers three main debugging methods: Verbose Mode, Debug Mode, and LangSmith Tracing.
    *   LangSmith provides the most comprehensive debugging solution with a UI and persisted logs.
    *   Verbose mode prints key events in a chain, while debug mode provides full logging of all components.
    *  These debugging methods can be used in combination to provide a comprehensive view of the application.

This summary provides an overview of how to debug LLM applications in LangChain, drawing on the information in the sources, and our conversation history. It covers the key debugging methods, their features, and how they can be used to troubleshoot issues in your applications.


In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

tools = [TavilySearchResults(max_results=1)]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'The 2023 film "Oppenheimer" was directed by Christopher Nolan.\n\nTo calculate Christopher Nolan\'s age in days, we need his birthdate. Christopher Nolan was born on July 30, 1970. \n\nGiven that today\'s date is October 30, 2023, we can calculate his age in days:\n\n1. Calculate the total number of days from July 30, 1970, to October 30, 2023.\n2. Count the complete years first and then the additional days.\n\nLet\'s do that calculation:\n\n- From July 30, 1970, to July 30, 2023, is 53 years.\n- From July 30, 2023, to October 30, 2023, is 3 months.\n\nNow, calculating the days:\n- 53 years = 53 * 365 = 19,445 days (not accounting for leap years).\n- There are 13 leap years between 1970 and 2023 (1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020).\n- Therefore, we add 13 days for leap years.\n\nTotal days from July 30, 1970, to July 30, 2023:\n- 19,445 + 13 = 19,458

In [3]:
from langchain.globals import set_verbose

set_verbose(True)
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Oppenheimer 2023 film director'}`


[{'url': 'https://www.imdb.com/title/tt15398776/fullcredits/', 'content': 'Oppenheimer (2023) cast and crew credits, including actors, actresses, directors, writers and more. Menu. ... director of photography: behind-the-scenes Jason Gary ... best boy grip ... film loader Luc Poullain ... aerial coordinator'}]
Invoking: `tavily_search_results_json` with `{'query': 'Oppenheimer 2023 film release date'}`


[{'url': 'https://www.rottentomatoes.com/m/oppenheimer_2023', 'content': "Genre:\nHistory,\nDrama,\nBiography\nOriginal Language:\nEnglish\nDirector:\nChristopher Nolan\nProducer:\nEmma Thomas,\nCharles Roven,\nChristopher Nolan\nWriter:\nChristopher Nolan\nRelease Date (Theaters):\nJul 21, 2023\nwide\nRelease Date (Streaming):\nNov 21, 2023\nBox Office (Gross USA):\n$328.1M\nRuntime:\n3h 0m\nDistributor:\nUniversal Pictures\nProduction Co:\nGadget Films,

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'The 2023 film "Oppenheimer" was directed by Christopher Nolan. \n\nTo calculate Christopher Nolan\'s age in days, we need his birthdate. Christopher Nolan was born on July 30, 1970. \n\nNow, let\'s calculate his age in days from July 30, 1970, to today. Today is October 5, 2023.\n\n1. Calculate the number of full years: \n   - From July 30, 1970, to July 30, 2023, is 53 years.\n  \n2. Calculate the remaining days from July 30, 2023, to October 5, 2023:\n   - August has 31 days: 31 - 30 = 1 day in August.\n   - September has 30 days: 30 days in September.\n   - October has 5 days: 5 days in October.\n\nAdding these together:\n- 1 (August) + 30 (September) + 5 (October) = 36 days.\n\nNow, converting the years to days:\n- 53 years = 53 * 365 = 19,345 days.\n- Account for leap years from 1970 to 2023. There are 13 leap years in this period (1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 20

In [4]:
from langchain.globals import set_debug

set_debug(True)
set_verbose(False)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age in days?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiti

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'The 2023 film "Oppenheimer" was directed by Christopher Nolan. \n\nTo calculate Christopher Nolan\'s age in days, we need his birthdate. Christopher Nolan was born on July 30, 1970. As of today, which is October 20, 2023, he would be:\n\n1. Calculate the total days from July 30, 1970 to October 20, 2023.\n\nLet\'s do the calculation:\n\n- From July 30, 1970 to July 30, 2023 is 53 years.\n- From July 30, 2023 to October 20, 2023 is 81 days.\n\nNow, converting 53 years into days:\n53 years = 53 * 365 = 19,445 days (we need to add the leap years).\n\nFrom 1970 to 2023, the leap years are: 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020.\nThat gives us 13 leap years.\n\nTotal days = 19,445 + 13 + 81 = 19,539 days.\n\nSo, Christopher Nolan is currently 19,539 days old.'}